In [59]:
#Import modules and dependencies
import cv2
import mediapipe as mp
import numpy as np
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils


In [60]:
#Calculate angle
def calculate_angle(a,b,c):
    a=np.array(a)
    b=np.array(b)
    c=np.array(c)

    radians=np.arctan2(c[1]-b[1],c[0]-b[0])-np.arctan2(a[1]-b[1],a[0]-b[0])
    angle=np.abs(radians*180.0/np.pi)

    if angle>180.0:
        angle=360-angle
    return angle

In [61]:

#Curl counter 
def start_counting_curls():
    counter=0
    stage=""

    cam = cv2.VideoCapture("Curl exercise.mp4")

    if not cam.isOpened():
        print("Error: Could not open webcam.")
        return 
    
    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        
        while cam.isOpened():
            ret, frame = cam.read()
            if not ret:
                break

            # Recolor image from BGR to RGB
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False  # Saves memory

            # Make detection
            results = pose.process(image)

            # Back to BGR from RGB (CV2 can only handle BGR format)
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

             # Extract landmarks and calculate angle
            if results.pose_landmarks:
                try:
                    # Get coordinates of shoulder, elbow, wrist
                    landmarks = results.pose_landmarks.landmark
                    shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x * image.shape[1],
                                landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y * image.shape[0]]
                    elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x * image.shape[1],
                             landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y * image.shape[0]]
                    wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x * image.shape[1],
                             landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y * image.shape[0]]

                    # Calculate angle
                    angle = int(calculate_angle(shoulder, elbow,wrist))

                    # Convert elbow coordinates to pixel values
                    elbow_x = int(elbow[0])
                    elbow_y = int(elbow[1])

                    # Visualize angle at the elbow location
                    #cv2.putText(image, str(angle), (elbow_x, elbow_y),cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

                    
                    #Curl counter
                    if angle>160:
                        stage="down"
                    if angle<40 and stage=="down":     
                        stage="up"
                        counter+=1
                        #print(counter)
                     

                except Exception as e:
                    print(f"Error processing landmarks: {e}")

                #Render Curl counter
                cv2.rectangle(image,(0,0),(225,73),(245,117,16),-1)
                cv2.putText(image,str(counter)+" "+str(stage),(10,50), cv2.FONT_HERSHEY_SIMPLEX,1, (255, 255, 255), 2, cv2.LINE_AA)
                             
                              

                # Render detections
                #mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                   #       mp_drawing.DrawingSpec(color=(250, 0, 0), thickness=2, circle_radius=2),  # style of nodes
                                    #      mp_drawing.DrawingSpec(color=(0, 250, 0), thickness=2, circle_radius=2))  # style of connections

            else:
                print("No landmarks detected.")

            # Display the frame
            cv2.imshow('Pose Detection', image)

            if cv2.waitKey(10) & 0xFF == ord('q'):
                break

    cam.release()
    cv2.destroyAllWindows()
    
    return counter


In [62]:
start_counting_curls()

4

In [63]:

#Squats counter 

def start_counting_squats():
    counter=0
    stage=""

    cam = cv2.VideoCapture("Squats exercise.mp4")

    if not cam.isOpened():
        return 
        
    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        while cam.isOpened():
            ret, frame = cam.read()
            if not ret:
                break
            # Recolor image from BGR to RGB
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False  # Saves memory

            # Make detection
            results = pose.process(image)

            # Back to BGR from RGB (CV2 can only handle BGR format)
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

             # Extract landmarks and calculate angle
            if results.pose_landmarks:
                try:
                    # Get coordinates of shoulder, elbow, wrist
                    landmarks = results.pose_landmarks.landmark
                    hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x * image.shape[1],
                                landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y * image.shape[0]]
                    knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x * image.shape[1],
                             landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y * image.shape[0]]
                    ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x * image.shape[1],
                             landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y * image.shape[0]]

                    # Calculate angle
                    angle = int(calculate_angle(hip, knee,ankle))

                    # Convert elbow coordinates to pixel values
                    knee_x = int(knee[0])
                    knee_y = int(knee[1])

                    # Visualize angle at the elbow location
                    #cv2.putText(image, str(angle), (knee_x, knee_y),cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

                    #Curl counter
                    if angle<90:
                        stage="down"
                    if angle>160 and stage=="down":     
                        stage="up"
                        counter+=1
                        #print(counter)
                    

                except Exception as e:
                    print(f"Error processing landmarks: {e}")

                #Render Curl counter
                cv2.rectangle(image,(0,0),(225,73),(245,117,16),-1)
                cv2.putText(image,str(counter)+" "+str(stage),(10,50), cv2.FONT_HERSHEY_SIMPLEX,1, (255, 255, 255), 2, cv2.LINE_AA)
                             
                              

                # Render detections
                #mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                 #                         mp_drawing.DrawingSpec(color=(250, 0, 0), thickness=2, circle_radius=2),  # style of nodes
                  #                        mp_drawing.DrawingSpec(color=(0, 250, 0), thickness=2, circle_radius=2))  # style of connections

            else:
                print("No landmarks detected.")

            # Display the frame
            cv2.imshow('Pose Detection', image)

            if cv2.waitKey(10) & 0xFF == ord('q'):
                break

    cam.release()
    cv2.destroyAllWindows()
    
    return counter


In [64]:
start_counting_squats()

5

In [65]:

def calculate_distance(point1, point2):
    return np.linalg.norm(np.array(point1) - np.array(point2))


#Lunges counter 
def start_counting_lunges():
    
    MIN_ANKLE_DT=50 #Assumption
    counter=0
    stage=""

    cam = cv2.VideoCapture("LUNGS.mp4")

    if not cam.isOpened():
        return 

    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        while cam.isOpened():
            ret, frame = cam.read()
            if not ret:
                break
                
            # Recolor image from BGR to RGB
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False  # Saves memory

            # Make detection
            results = pose.process(image)

            # Back to BGR from RGB (CV2 can only handle BGR format)
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

             # Extract landmarks and calculate angle
            if results.pose_landmarks:
                try:
                    # Get coordinates of shoulder, elbow, wrist
                    landmarks = results.pose_landmarks.landmark
                    right_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x * image.shape[1],
                                landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y * image.shape[0]]
                    right_knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x * image.shape[1],
                             landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y * image.shape[0]]
                    right_ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x * image.shape[1],
                             landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y * image.shape[0]]
                    right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x * image.shape[1],
                             landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y * image.shape[0]]
                    left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x * image.shape[1],
                                landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y * image.shape[0]]
                    left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x * image.shape[1],
                                landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y * image.shape[0]]
                    left_knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x * image.shape[1],
                             landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y * image.shape[0]]
                    left_ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x * image.shape[1],
                             landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y * image.shape[0]]

                    # Calculate angle
                    right_knee_angle = int(calculate_angle(right_hip, right_knee,right_ankle))
                    left_knee_angle = int(calculate_angle(left_hip, left_knee,left_ankle))
                    right_hip_angle = int(calculate_angle(right_shoulder,right_hip, right_knee))
                    left_hip_angle = int(calculate_angle(left_shoulder,left_hip, left_knee))


                    # Convert knee coordinates to pixel values
                    left_knee_x = int(left_knee[0])
                    left_knee_y = int(left_knee[1])
                    right_knee_x = int(right_knee[0])
                    right_knee_y = int(right_knee[1])

                    left_hip_x = int(left_hip[0])
                    left_hip_y = int(left_hip[1])
                    right_hip_x = int(right_hip[0])
                    right_hip_y = int(right_hip[1])


                    #Calculate distance between Ankles
                    ankle_dt=calculate_distance(left_ankle,right_ankle)

                    # Visualize angle at the elbow location
                    #cv2.putText(image, str(left_knee_angle), (left_knee_x, left_knee_y),cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
                    #cv2.putText(image, str(right_knee_angle), (right_knee_x-50, right_knee_y-50),cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

                    #cv2.putText(image, str(left_hip_angle), (left_hip_x-100, left_hip_y-100),cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
                    #cv2.putText(image, str(right_hip_angle), (right_hip_x, right_hip_y),cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
                    

                    #Angle
                    if right_hip_angle>150 and left_hip_angle>150 and right_knee_angle>150 and left_knee_angle>150:
                        stage="up"
                    
                    if stage=="up" and right_knee_angle<100 and left_knee_angle<100 and ankle_dt>=MIN_ANKLE_DT:
                        stage="down"
                        counter+=1
                        #print(counter)

                    #print(stage,right_hip_angle,left_hip_angle,right_knee_angle,left_knee_angle,ankle_dt)

                except Exception as e:
                    print(f"Error processing landmarks: {e}")

                #Render Curl counter
                cv2.rectangle(image,(0,0),(225,73),(245,117,16),-1)
                cv2.putText(image,str(counter)+" "+str(stage),(10,50), cv2.FONT_HERSHEY_SIMPLEX,1, (255, 255, 255), 2, cv2.LINE_AA)
                             
                # Render detections
                #mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                 #                         mp_drawing.DrawingSpec(color=(250, 0, 0), thickness=2, circle_radius=2),  # style of nodes
                  #                        mp_drawing.DrawingSpec(color=(0, 250, 0), thickness=2, circle_radius=2))  # style of connections

            else:
                print("No landmarks detected.")

            # Display the frame
            cv2.imshow('Pose Detection', image)

            if cv2.waitKey(10) & 0xFF == ord('q'):
                break

    cam.release()
    cv2.destroyAllWindows()
    
    return counter


In [66]:
start_counting_lunges()

6

In [67]:

#Pull UPS counter

def start_counting_pullups():

    counter=0
    stage=""

    cam = cv2.VideoCapture("Pull UP.mp4")

    if not cam.isOpened():
        return 

    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        while cam.isOpened():
            ret, frame = cam.read()
            if not ret:
                break

            # Recolor image from BGR to RGB
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False  # Saves memory

            # Make detection
            results = pose.process(image)

            # Back to BGR from RGB (CV2 can only handle BGR format)
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

            
            # Extract landmarks and calculate angle
 
            if results.pose_landmarks:
                try:
                    # Get coordinates of shoulder, elbow, wrist
                    landmarks = results.pose_landmarks.landmark
                    shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x * image.shape[1],
                                landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y * image.shape[0]]
                    elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x * image.shape[1],
                             landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y * image.shape[0]]
                    wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x * image.shape[1],
                             landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y * image.shape[0]]

                    # Calculate angle
                    angle = int(calculate_angle(shoulder, elbow,wrist))

                    # Convert elbow coordinates to pixel values
                    elbow_x = int(elbow[0])
                    elbow_y = int(elbow[1])

                    # Visualize angle at the elbow location
                    #cv2.putText(image, str(angle), (elbow_x, elbow_y),cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

                    
                    #Curl counter
                    if angle>160:
                        stage="down"
                    if angle<40 and stage=="down":     
                        stage="up"
                        counter+=1
                        #print(counter)
                     

                except Exception as e:
                    print(f"Error processing landmarks: {e}")

                #Render Curl counter
                cv2.rectangle(image,(0,0),(225,73),(245,117,16),-1)
                cv2.putText(image,str(counter)+" "+str(stage),(10,50), cv2.FONT_HERSHEY_SIMPLEX,1, (255, 255, 255), 2, cv2.LINE_AA)
                             
                              

                # Render detections
                #mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                 #                         mp_drawing.DrawingSpec(color=(250, 0, 0), thickness=2, circle_radius=2),  # style of nodes
                  #                        mp_drawing.DrawingSpec(color=(0, 250, 0), thickness=2, circle_radius=2))  # style of connections

            else:
                print("No landmarks detected.")

            # Display the frame
            cv2.imshow('Pose Detection', image)

            if cv2.waitKey(10) & 0xFF == ord('q'):
                break

    cam.release()
    cv2.destroyAllWindows()
    
    return counter


In [68]:
start_counting_pullups()

4

In [69]:
def play_video(filename):

    cam = cv2.VideoCapture(filename)

    if not cam.isOpened():
        print("Error: Could not open webcam.")
        return
    
    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        while cam.isOpened():
            ret, frame = cam.read()
            if not ret:
                break

            # Recolor image from BGR to RGB
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False  # Saves memory

            # Back to BGR from RGB (CV2 can only handle BGR format)
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

            cv2.imshow('Pose Detection', image)

            if cv2.waitKey(10) & 0xFF == ord('q'):
                break

    cam.release()
    cv2.destroyAllWindows()

In [70]:
print("Counting Curls...")
play_video("Start Biceps.mp4")
print("Curls count:",start_counting_curls())

print("Counting Squates...")
play_video("Start Squates.mp4")
print("Squates count",start_counting_squats())

print("Counting Lunges...")
play_video("Start lunges.mp4")
print("Lunges count",start_counting_lunges())

print("Counting Pull-Ups...")
play_video("Start pullups.mp4")
print("Pull-Ups count",start_counting_pullups())

print("Well done...")

Counting Curls...
Curls count: 4
Counting Squates...
Squates count 5
Counting Lunges...
Lunges count 6
Counting Pull-Ups...
Pull-Ups count 4
Well done...
